Цель: Используя файл ab_data.csv проанализировать, есть ли улучшение в новой landing page сайта по сравнению со старой

In [80]:
import pandas as pd
import numpy as np

import scipy
import statsmodels.api as sm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('data/ab_data.csv', header = 0)

In [3]:
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


#### EDA

In [4]:
data.shape

(294478, 5)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [11]:
# no of duplicates in userid
data.shape[0] - data.user_id.nunique()

3894

In [12]:
data.drop_duplicates(subset='user_id', keep='first', inplace=True)

In [13]:
data.shape[0] - data.user_id.nunique()

0

In [14]:
data.group.value_counts()

treatment    145352
control      145232
Name: group, dtype: int64

In [15]:
data.landing_page.value_counts()

new_page    145320
old_page    145264
Name: landing_page, dtype: int64

Странно, что всегда treatment == new_page не соблюдается. 

In [16]:
target = 'converted'

In [17]:
data[target].mean()

0.1195695564793657

In [30]:
data.groupby(['group', 'landing_page']).count()[target]

group      landing_page
control    new_page          1006
           old_page        144226
treatment  new_page        144314
           old_page          1038
Name: converted, dtype: int64

In [59]:
data.groupby(['group', 'landing_page']).converted.mean()

group      landing_page
control    new_page        0.121272
           old_page        0.120290
treatment  new_page        0.118727
           old_page        0.134875
Name: converted, dtype: float64

In [26]:
data.loc[(data['group'] =='control') & (data['landing_page'] =='new_page')].head()

,user_id,timestamp,group,landing_page,converted
22,767017,2017-01-12 22:58:14.991443,control,new_page,0
240,733976,2017-01-11 15:11:16.407599,control,new_page,0
490,808613,2017-01-10 21:44:01.292755,control,new_page,0
846,637639,2017-01-11 23:09:52.682329,control,new_page,1
850,793580,2017-01-08 03:25:33.723712,control,new_page,1


In [27]:
data.loc[(data['group'] =='treatment') & (data['landing_page'] =='old_page')].head()

,user_id,timestamp,group,landing_page,converted
308,857184,2017-01-20 07:34:59.832626,treatment,old_page,0
327,686623,2017-01-09 14:26:40.734775,treatment,old_page,0
357,856078,2017-01-12 12:29:30.354835,treatment,old_page,0
685,666385,2017-01-23 08:11:54.823806,treatment,old_page,0
713,748761,2017-01-10 15:47:44.445196,treatment,old_page,0


##### ACHTUNG: Теперь берем только df, он соответствует критериям, а не data

In [31]:
df =  data.loc[(data['group'] =='treatment') & (data['landing_page'] =='new_page')]

In [35]:
df = df.append(data.loc[(data['group'] =='control') & (data['landing_page'] =='old_page')])

In [36]:
df.tail()

,user_id,timestamp,group,landing_page,converted
294471,718310,2017-01-21 22:44:20.378320,control,old_page,0
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [56]:
counts = df.groupby(['group', 'landing_page']).converted.count()
print(counts)

group      landing_page
control    old_page        144226
treatment  new_page        144314
Name: converted, dtype: int64


In [58]:
sums = df.groupby(['group', 'landing_page']).converted.sum()
print(sums)

group      landing_page
control    old_page        17349
treatment  new_page        17134
Name: converted, dtype: int64


In [54]:
averages = df.groupby(['group','landing_page']).converted.mean()
print(averages)

print('The difference is: {:.2f}%'.format((averages[0] - averages[1])*100))

group      landing_page
control    old_page        0.120290
treatment  new_page        0.118727
Name: converted, dtype: float64
The difference is: 0.16%


Прежняя конверсия 0.12, а с новой страницой стало чуть хуже 0.11. Следует понять 0,16% **статистически значимо или нет**.


$H_0\colon$ $p_{new}$ = $p_{old}$.

$H_1\colon$ $p_{new}$ $\neq$ $p_{old}$.

In [75]:
n_rows_a = counts[0]
n_rows_b = counts[1]

convert_a = sums[0]
convert_b = sums[1]

p_a = averages[0]
p_b = averages[1]

In [76]:
p_a , p_b

(0.12029037760181936, 0.11872721981235362)

In [89]:
interval = 1.96 * np.sqrt((p_a*(1-p_a)/n_rows_a) + (p_b*(1-p_b)/n_rows_b))
print("95% CI Lower bond: {0:.3f}%, upper bond: {1:.3f}%".format(p_b-p_a-interval, p_b-p_a+interval))

95% CI Lower bond: -0.004%, upper bond: 0.001%


В этот интервал входит 0, то есть наша гипотеза $H_0$. Похоже c уровнем значимости 0.05, интервал включает нуль, поэтому у нас **нет достаточных данных**, чтобы сказать, что есть значимая разница между группами a,b.

In [97]:
z_score, p_value = sm.stats.proportions_ztest([convert_a, convert_b], 
                                              [n_rows_a, n_rows_b], 
                                              alternative='larger')
print("Z-score={0:.3f},  p_value={1:.3f}".format(z_score, p_value))

Z-score=1.294,  p_value=0.098


In [99]:
z_score >= 1.96

False

Z_score не выходит за рамки отказа $H_0$

In [100]:
p_value > 0.05

True

Мы не можем отвергнуть $H_0$. При частоте ошибок типа I, равной 5%, если p_value больше него, то мы не можем отвергнуть $H_0$. Старые страницы имеют более высокую или равную конверсию, чем новые с 90% вероятностью.

**Статистически значимого улучшения при внедрении новой страницы не произошло.**

In [98]:
(1-p_value)*100

90.22081785605928